
<div class="alert alert-info">
    
I included everything in the first notebook you provided. 

    
</div>


<div style="border: 5px solid purple; padding: 15px; margin: 5px">
<b> Reviewer's comment</b>
    
Hi Brett ! My name is Svetlana (my handle on Discord is `svetatripleten`). Congratulations on submitting Computer Vision project! 🎉
    

> I included everything in the first notebook you provided.


Which notebook? Can you upload it please? 
</div>

<div class="alert alert-info">
    
I am a little confused - TTT provided two notebooks: one for the EDA, and another for the model training (which is this one). Each notebook had identical precode, so I just did everything in the first notebook that was there for the EDA. I will just copy and paste everything I did in that notebook to this one I guess. 

</div>


<div class="alert alert-info">
    
I wanted to actually train a CNN instead of just copy and pasting the ouputs from the model you guys trained. For this reason I downloaded the data set provided by ChaLearn.

The original data set already had different folders for training data, validation data, and testing data. Hence, the code to load the data is different (i.e. three sets instead of one). I did not want to run an EDA on data that was different than the data I trained the model on. Therefore, I have provided the code as I ran it on the Jupyter Notebook I launched from VS code on my local device. The This means that I loaded three different sets of data. 
    
Three other points which are worth noting: 

    1. Since ImageDataGenerator is deprecated I chose to design a function to accomplis the same tasks.
    2. I ran the EDA in the Jupyter Notebook I launcehd from VS code, but I trained the model using A100 GPUs on google colab. I did it this way because there was no need to waste extra time and memory re-plotting the images every time I had to re-run all the cells to train the model again.
    3. The code below includes code that I ran for the EDA in my local notebook first, followed by the code I used to train the model in google colab, with the exception of the load_csv() function which appears first here, but was utilized in my colab notebook not the Jupyter Notebook I used for the EDA. 
    
If any of this is a problem feel free to send the project back, as it will not be too much trouble to run the EDA on the dataset the way it constructed on TTT, and to use ImageDataGenerator. 

    
</div>

<div style="border: 5px solid purple; padding: 15px; margin: 5px">
<b> Reviewer's comment 2</b>
    


Understood. It totally makes sense! Actually, you are probably going to use PyTorch much more often. It is more flexible and preferable.


However, I do not have the data you used, so I cannot run your code. I reviewed the code without running it, and everything looks correct. I do not see any significant issues. Great job!  
     

I used the standard color marking:
    
  
    
<div style="border: 5px solid green; padding: 15px; margin: 5px">

Great solutions and ideas that can and should be used in the future are in green comments. Some of them are: 
    
    
- You have successfully displayed and analyzed age distribution, great!
    
    
- It's also a good idea to display several photos, since we need to understand what kind of data we have;

    
- You have correctly built a model; 


- Analyzed the result in the final conclusion, which is also important! The analysis usually helps us to identify the area for further improvement.

</div>
    
<div style="border: 5px solid gold; padding: 15px; margin: 5px">
<b> Reviewer's comment </b>

Yellow color indicates what should be optimized. This is not necessary, but it will be great if you make changes to this project. 
 
</div>
<div style="border: 5px solid red; padding: 15px; margin: 5px">
<b> Reviewer's comment </b>
    
Issues that must be corrected to achieve accurate results are indicated in red comments. However, there are no such issues, well done!

</div>
<hr>
    
<font color='dodgerblue'>**To sum up:**</font> thank you very much for submitting the project! You have successfully built a model and recieved significant results, great job! Your project is great and does not have any issues that need to be fixed. Therefore, I accept the project now.


<hr>
    
    
✍️ Here's a nice playlist [Introduction to
Deep Learning](https://www.youtube.com/playlist?list=PLtBw6njQRU-rwp5__7C0oIVt26ZgjG9NI) on youtube that you may find helpful and interesting. This is another interesting [Computer Vision Playlist](https://www.youtube.com/playlist?list=PLf7L7Kg8_FNxHATtLwDceyh72QQL9pvpQ).     
 
    
    
Here's a link to [AI for beginners](https://github.com/microsoft/ai-for-beginners) course by Microsoft. 
    
<hr>
    
I hope you enjoyed this topic. Good luck! 
</div>

# Building a Convolutional Neural Network (CNN) to Help Good Seed Grocery Store Identify Underage Customers Attempting to Buy Alcohol

Good Seed Grocery Store is looking for an image recognition model that can help them avoid selling alcohol to underage customers. The goal of this project is to create a CNN which can aid them in this task. We will start by undergoing an exploratory data analysis to make sure the data is ready to be fed to the model. Then we will train, validate, and test a model comprised of a ResNet50 layer, a GlobalAveragePooling2D layer, and lastly a fully connected Dense layer without one output neuron that utilizes a relu activation function (relu is ideal for age prediction because it can only output positive values). We may make some subtle adjustements to this approach if we can see that they yield improvd performance. 

## Initialization

In [ ]:
# Standard library imports
import os                              # File and path operations
from collections import Counter

# Third-party imports
import numpy as np                     # Numerical computations
import pandas as pd                    # Data handling and analysis
import matplotlib.pyplot as plt       # Plotting
import seaborn as sns                  # Enhanced plotting
import tensorflow as tf                # TensorFlow for deep learning
from PIL import Image                 # Pillow for image file handling

## Load Data

The dataset is stored in the `/datasets/faces/` folder, there you can find
- The `final_files` folder with 7.6k photos
- The `labels.csv` file with labels, with two columns: `file_name` and `real_age`

Given the fact that the number of image files is rather high, it is advisable to avoid reading them all at once, which would greatly consume computational resources. We recommend you build a generator with the ImageDataGenerator generator. This method was explained in Chapter 3, Lesson 7 of this course.

The label file can be loaded as an usual CSV file.

In [ ]:
# Load CSVs containing real age and file names
def load_csv(base_path):
    train_df = pd.read_csv(os.path.join(base_path, 'gt_avg_train.csv'))
    val_df = pd.read_csv(os.path.join(base_path, 'gt_avg_valid.csv'))
    test_df = pd.read_csv(os.path.join(base_path, 'gt_avg_test.csv'))
    return train_df, val_df, test_df

base_path = '/content/drive/MyDrive'  # change here if needed per environment
train_df, val_df, test_df = load_csv(base_path)

In [ ]:
print(len(train_df))
print(len(val_df))
print(len(test_df))

## EDA

In [ ]:
# Create a function that joins folder and fname, 
# while also allowing the option to use the cropped images. 
def get_file_path(base_path, folder, fname, use_cropped_faces=True):
    if use_cropped_faces:
        fname = fname.replace('.jpg', '.jpg_face.jpg')
    rturn os.path.join(base_path, folder, fname)


# Calls the get_file_path function on the 'file_name' column of each DataFrame
train_df['file_path'] = train_df['file_name'].apply(lambda x: get_file_path('train', x))
val_df['file_path']   = val_df['file_name'].apply(lambda x: get_file_path('valid', x))
test_df['file_path']  = test_df['file_name'].apply(lambda x: get_file_path('test', x))

In [ ]:
# Get a look at the DataFrames with the added 'file_path' column
print(train_df.head())
print(val_df.head())
print(test_df.head())

In [ ]:
# Define a function that will load the data set iteratively (i.e. not all at once)
# and maximizes efficiency by utilizing pre-fetching. 
def load_data(df, batch_size=32, shuffle=True):
    file_paths = df['file_path'].values
    labels = df['real_age'].values

    dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))

    def load_and_preprocess(path, label):
        image = tf.io.read_file(path)
        image = tf.image.decode_jpeg(image, channels=3)         # Decode image
        image = tf.image.resize(image, [224, 224])              # Resize to uniform size
        image = image / 255.0                                   # Normalize to [0, 1]
        return image, label

    dataset = dataset.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    
    image_shape = next(iter(dataset.batch(1).as_numpy_iterator()))[0].shape

    if shuffle:
        # Shuffle data set at start of each epoch (assuming shuffle=True)
        dataset = dataset.shuffle(buffer_size=1000)

    # Fetch a batch size of 32
    dataset = dataset.batch(batch_size)
    
    # Prefetch the next set while the model is traniing on current set
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset, image_shape

<div style="border: 5px solid gold; padding: 10px; margin: 5px">
<b>   Reviewer's comment 2</b>
    
We rarely need to define a function inside another function. It does not make the code easier to understand nor does it simplify the debugging process.  

</div>

In [ ]:
# Assign iterative output of build_data set function to relevant data set variable
train_ds, image_shape = load_data(train_df, batch_size=32, shuffle=True)
val_ds, _ = load_data(val_df, batch_size=32, shuffle=False)
test_ds, _ = load_data(test_df, batch_size=32, shuffle=False)

We are really only interested in the 'real_age' column. Therefore, the real_age column will be the focus of our exploratory data analysis.

In [ ]:
# Instruct matplotlib to use seaborn presentation style
plt.style.use('seaborn-v0_8')
sns.set_theme(style='whitegrid')

In [ ]:
# Concatinate lbabls data sets for global EDA analysis
eda_df = pd.concat([train_df, val_df, test_df], ignore_index=True)

In [ ]:
print(eda_df.info())
eda_df.head()

In [ ]:
eda_grouped_df = eda_df.groupby('real_age').sum().reset_index()

In [ ]:
eda_grouped_df

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)

sns.boxplot(data=eda_grouped_df, x='real_age', color='lightblue')
plt.title('Box plot of Age')
plt.xlabel('Age')

plt.subplot(1, 2, 2)
sns.histplot(data=eda_df, x='real_age', bins=100, kde=True, color='lightgreen')
plt.xticks(ticks=range(0, 101, 5))
plt.title('Histogram of Age')
plt.xlabel('Age')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

Our data set is a fair representation of the distribution of age in the United States. For example, as of 2020 the most numerous age range in the United States is 15-34 which roughly corresponds to what we see in our data set. The next most common age range in the United States after 15-34 is 55-59. This is not quite reflected in our data set. In our data the most common age after any between 15 and 34 is 40, and next is 50. Given that the paper associated with this data wsa published in 2017, it is fair to think that this data was taken in 2017 or a little bit before. If taken in 2016, depending on how the months align, this could put the age 50 peak at age 55 by some month in 2020. Even still, our data set is not in perfect alignment with that of the US population in 2020, and likely does not reflect the US age population in 2025.

Nevertheless, for the task at hand, our data set is appropriate. We are looking to predict the ages of people to decide if they are old enough to buy alcohol. Therefore, we want the most data points for people who are likely to buy aclohol when they are too young, and people who are old enough to buy alcohol but young looking enough so that their age appropriateness is not immediately obvious. The two most common ages in our data set are 25 and 30, with most of the next most common ages appearing between 16 and 30.

I got my information about the age distribution for the United States in 2020 here

Now let's get a look at the training, validation, and test sets separately to make sure their distributions for the real_age column are similar.

In [ ]:
# Create list of names and DataFrames for easy plotting with for loops
names = ['Train', 'Validation', 'Test']
df_list = [train_df, val_df, test_df]

In [ ]:
# Plot target variable for each set for piecewise EDA 
plt.figure(figsize=(15, 4))
for i, (df, name) in enumerate(zip(df_list, names)):
    plt.subplot(1, 3, i+1)
    sns.boxplot(data=df, x='real_age', color='lightblue')
    plt.title(f'Box plot of Age ({name} set)')
    plt.xlabel('Age')
plt.tight_layout()
plt.show

plt.figure(figsize=(15, 4))
for i, (df, name) in enumerate(zip(df_list, names)):
    plt.subplot(1, 3, i + 1)
    sns.histplot(data=df, x='real_age', bins=100, kde=True, color='lightgreen')
    plt.xticks(ticks=range(0, 101, 5))
    plt.title(f'Histogram of Age ({name} set)')
    plt.xlabel('Age')
    plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

The distribution of the target variable between our training set, validation set, and test set are very similar. Therefore, our model should generalize well to the validation and test sets. Let's get a look at the data set in full. Next we will print out some images to make sure there are no obvious misalignments between the labels and the images. 

In [ ]:
for images, labels in train_ds.take(1):
    # Select first 15 images and corresponding labels
    images = images[:15]
    labels = labels[:15]
    break

# Plot the images with corresponding labels
plt.figure(figsize=(15, 6))
for i in range(len(images)):
    plt.subplot(3, 5, i+1)
    plt.imshow(images[i].numpy()) 
    plt.title(f"Age: {int(labels[i])}")
    plt.axis('off')

plt.tight_layout()
plt.show()

The ages for our faces all appear to be appropriate, and we hae a variety of situations and lightings between images. There are no obvious biases presening themselves that would lead to overfitting. Therefore, we will leave the images as they are. Let's give the extremes of our data set a look just to make sure nothing is off with the age labels.

In [ ]:
# Create variables containing images and lables for youngest 5
# faces and oldest 5 faces
youngest = eda_df.nsmallest(5, 'real_age')
oldest = eda_df.nlargest(5, 'real_age')

In [ ]:
# Define function to plot youngest and oldest faces
def show_images(df, title):
    plt.figure(figsize=(15, 3))
    # itreate over each row for access to index and row contents
    for i, row in enumerate(df.itertuples()):
        
        img = tf.io.read_file(row.file_path)           # Extract file path
        img = tf.image.decode_jpeg(img, channels=3)    # Decode image found at file path
        img = tf.image.resize(img, [224, 224])/255.0   # rescale image 

        plt.subplot(1, len(df), i+1) # We use i+1 because subplot index starts at 1, and i starts at 0. 
        plt.imshow(img.numpy())
        plt.title(f"Age: {int(row.real_age)}")
        plt.axis('off')
    plt.suptitle(title)
    plt.tight_layout()
    plt.plot()

In [ ]:
show_images(youngest, 'Youngest Faces')
show_images(oldest, 'Oldest Faces')

Okay, everything looks good here. Let's check the original image sizes to make sure the dimensions we rescaled to are appropriate. 

In [ ]:
%%time
size_counter = Counter()
for path in eda_df['file_path']:
    try:
        with Image.open(path) as img:   # Use 'with' to presece resources
            # Count the number of images with a given size - Counter() works by using the tuple as a key
            size_counter[img.size] +=1
    except Exception as e:
        print("Error with {path}: {e}")

print(size_counter.most_common(10))

Great! All of our images are square, and there are no significant outliers. Therefore, our rescaling to 224, 224 is appropriate. Now we can move on to building and training our neural network.

## Modelling

Define the necessary functions to train your model on the GPU platform and build a single script containing all of them along with the initialization section.

To make this task easier, you can define them in this notebook and run a ready code in the next section to automatically compose the script.

The definitions below will be checked by project reviewers as well, so that they can understand how you built the model.

In [ ]:
def create_model(input_shape):

  """
  It defines model
  """

  backbone = ResNet50(weights='imagenet',
                      input_shape=input_shape,
                      include_top=False)

  model = Sequential()
  model.add(backbone)
  model.add(GlobalAveragePooling2D())
  model.add(Dense(1, activation='relu'))

  optimizer = Adam(learning_rate=0.0005)
  model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

  return model

<div style="border: 5px solid green; padding: 15px; margin: 5px">
<b>   Reviewer's comment 2</b>


Correct! 
    
    
- `Dense(1, activation='relu')` is appropriate, since you are predicting a single value. Using 1 neuron makes sense here.
    
    
- Adam optimizer with learning rate 0.0005 is a solid and commonly used choice, well done. 


- The use of MSE loss and MAE as metric is also correct. MSE encourages precision, and MAE helps us interpret error scale in the same units as the target.

</div>

In [ ]:
def train_model(model, train_data, val_data, batch_size=None, epochs=20,
                steps_per_epoch=None, validation_steps=None):

  """
  Trains the model given the parameters
  """
  if steps_per_epoch is None:
    steps_per_epoch = len(train_data)

  if validation_steps is None:
    validation_steps = len(val_data)

  checkpoint = ModelCheckpoint( # Choose the parameters that performed best on validation set
      filepath='best.weights.h5',
      monitor='val_loss',
      save_best_only=True,
      save_weights_only=True)

  model.fit(train_data,
            validation_data=val_data,
            batch_size=batch_size, epochs=epochs,
            steps_per_epoch=steps_per_epoch,
            validation_steps=validation_steps,
            callbacks=[checkpoint],
            verbose=2)
  
  return model

In [ ]:
def model_eval(model, test_target, human_estimates_avg):
    model_mse, model_mae = model.evaluate(test_ds)
    mae_humans = mean_absolute_error(test_target, human_estimates_avg)
    mse_humans = mean_squared_error(test_target, human_estimates_avg)
    print(f"MAE for the model: {model_mae}")
    print(f"MSE for the model: {model_mse}")
    print(f"MAE human guess avg: {mae_humans}")
    print(f"MSE human guess avg: {mse_humans}")

## Prepare the Script to Run on the GPU Platform

Given you've defined the necessary functions you can compose a script for the GPU platform, download it via the "File|Open..." menu, and to upload it later for running on the GPU platform.

N.B.: The script should include the initialization section as well. An example of this is shown below.

In [2]:
# prepare a script to run on the GPU platform

# Define a function to clean zero-width characters 
def clean(text):
    return text.replace('\u200b', '')

# Define the import section as a string 
init_str = clean("""
# Core libraries
import os                              # File and path operations
import numpy as np                     # Numerical computations
import pandas as pd                    # Data handling and analysis
from tqdm import tqdm                  # Progress bars

# Scikit-learn (data metrics)
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Deep learning: TensorFlow + Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Dense, GlobalAveragePooling2D)
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

# Image processing
from PIL import Image                  # Pillow for image file handling
""")

# Write to a script file
with open('model.py', 'w') as f:
    
    # Write the import section
    f.write(init_str)
    f.write('\n\n')

    # Write the source code of required functions
    for fn_name in [load_csv, get_file_path, load_data, create_model, train_model, model_eval]:
        src = inspect.getsource(fn_name)
        f.write(clean(src))  # Clean function body too
        f.write('\n\n')

    # Write the main script logic
    main_script = clean("""
if __name__ == '__main__':
    base_path = '/content/drive/MyDrive'  # change here if needed per environment
    train_df, val_df, test_df = load_csv(base_path)

    train_df['file_path'] = train_df['file_name'].apply(lambda x: get_file_path(base_path, 'train', x))
    val_df['file_path'] = val_df['file_name'].apply(lambda x: get_file_path(base_path, 'valid', x))
    test_df['file_path'] = test_df['file_name'].apply(lambda x: get_file_path(base_path, 'test', x))

    train_ds, image_shape = load_data(train_df)
    val_ds, _ = load_data(val_df)
    test_ds, _ = load_data(test_df, shuffle=False)

    model = create_model(image_shape)
    train_model(model, train_ds, val_ds)

    model_eval(model, test_df['real_age'], test_df['apparent_age_avg'])
""")

    f.write(main_script)

NameError: name 'load_csv' is not defined

### Output

Place the output from the GPU platform as an Markdown cell here.

<div class="alert alert-info">
    
This is the output from the script I ran in google colab with A100 GPUs, not the output that was provided by the TTT platform, although I used the same model structure. The only difference with my model was that I used ModelCheckpoint() to leave the model with the parameters that performed best on the validation set during training. 

</div>


Epoch 1/20
129/129 - 382s - 3s/step - loss: 188.3276 - mae: 10.3624 - val_loss: 404.7741 - val_mae: 15.1994
Epoch 2/20
129/129 - 11s - 85ms/step - loss: 87.4498 - mae: 7.2506 - val_loss: 368.2730 - val_mae: 14.3708
Epoch 3/20
129/129 - 12s - 91ms/step - loss: 60.4758 - mae: 5.9903 - val_loss: 354.6083 - val_mae: 14.0227
Epoch 4/20
129/129 - 12s - 90ms/step - loss: 39.5704 - mae: 4.7526 - val_loss: 254.5890 - val_mae: 12.3463
Epoch 5/20
129/129 - 12s - 90ms/step - loss: 26.9462 - mae: 3.9868 - val_loss: 231.6930 - val_mae: 11.8536
Epoch 6/20
129/129 - 11s - 84ms/step - loss: 20.8511 - mae: 3.5217 - val_loss: 140.5745 - val_mae: 9.2435
Epoch 7/20
129/129 - 12s - 91ms/step - loss: 19.7864 - mae: 3.4079 - val_loss: 114.7309 - val_mae: 7.9518
Epoch 8/20
129/129 - 12s - 92ms/step - loss: 18.1562 - mae: 3.2287 - val_loss: 99.1776 - val_mae: 7.3540
Epoch 9/20
129/129 - 12s - 91ms/step - loss: 16.8994 - mae: 3.0925 - val_loss: 91.0080 - val_mae: 7.0221
Epoch 10/20
129/129 - 11s - 82ms/step - loss: 14.0554 - mae: 2.8471 - val_loss: 100.2822 - val_mae: 7.6958
Epoch 11/20
129/129 - 11s - 89ms/step - loss: 12.8367 - mae: 2.7504 - val_loss: 87.6442 - val_mae: 6.9339
Epoch 12/20
129/129 - 10s - 76ms/step - loss: 12.8009 - mae: 2.7349 - val_loss: 121.9496 - val_mae: 7.8613
Epoch 13/20
129/129 - 10s - 75ms/step - loss: 13.0961 - mae: 2.7409 - val_loss: 97.2528 - val_mae: 7.2666
Epoch 14/20
129/129 - 10s - 75ms/step - loss: 12.7429 - mae: 2.6085 - val_loss: 87.9459 - val_mae: 6.8886
Epoch 15/20
129/129 - 10s - 75ms/step - loss: 11.2712 - mae: 2.5433 - val_loss: 117.2740 - val_mae: 7.9199
Epoch 16/20
129/129 - 10s - 75ms/step - loss: 10.1002 - mae: 2.3800 - val_loss: 155.8389 - val_mae: 10.2878
Epoch 17/20
129/129 - 11s - 86ms/step - loss: 9.1066 - mae: 2.3045 - val_loss: 86.0609 - val_mae: 6.8119
Epoch 18/20
129/129 - 10s - 79ms/step - loss: 7.6095 - mae: 2.0997 - val_loss: 96.2923 - val_mae: 7.2657
Epoch 19/20
129/129 - 10s - 75ms/step - loss: 6.8119 - mae: 1.9936 - val_loss: 90.3570 - val_mae: 6.9667
Epoch 20/20
129/129 - 10s - 75ms/step - loss: 5.9435 - mae: 1.8791 - val_loss: 92.4066 - val_mae: 6.9797
<Sequential name=sequential, built=True>

<div style="border: 5px solid green; padding: 15px; margin: 5px">
<b> Reviewer's comment 2</b>
   

You can use Markdown cell for your code and results: 

    ```python
    
    # code
    
    
    ```
    
Example:
    
</div>


```python

import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50

...



Epoch 1/20
356/356 - 35s - loss: 95.3532 - mae: 7.4339 - val_loss: 124.3362 - val_mae: 8.4921  
```

Our output shows that our model shows significant improvement early on. However, the model did not show consistent improvement throughout training. I added utilized the ModelCheckpoint() method for this reason. 

Our MAE on the trianing set was much better than on the validation set. This suggests there is some overfitting going on. Our model may benefit from the addition of some regularization techniques. I ran a couple of iterations utilizing dropout with little to no benefit. However, perhaps some l2 regularization could be beneficial. 

In [ ]:
model_eval(model, test_df['real_age'], test_df['apparent_age_avg'])

model_eval output:
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 140.4368 - mae: 8.4980
MAE for the model: 8.538291931152344
MSE for the model: 141.1843719482422
MAE human guess avg: 4.58948056502737
MSE human guess avg: 41.15976503909733

## Conclusions

Our model did not perform quite as well as the humans who estimated the ages in the photos for the Appa-Real database. This is okay if the goal is to reduce the workload of individuals. The model still offers potential benefit in flagging customers who are potentially under age. It just cannot have the final say. Also, it can help weed out unreliable employees if it repeatedly predicts ages under 21 for people whom the cashier willingly sells alcohol to.

<div style="border: 5px solid green; padding: 15px; margin: 5px">
<b>   Reviewer's comment 2 </b>
    
    
We can still think about further improvements. Sometimes it is possible to increase the dataset. What else can be done? Sometimes it may be helpful to try different data augmentation approaches. By the way, here's a good library [albumentations](https://github.com/albumentations-team/albumentations). Apart from this, we can consider introducing additional layers with some normalization methods. The optimization part is good. Adam is quite flexible, but you can still try other optimizers if you want. For example, AdamW sometimes shows better results. 



Regarding age verification, I would argue that we have some noticable limits here. It is much more reliable and much easier to just compare the id or passport photo to the customer's face. 

</div>

# Checklist

- [ ]  Notebook was opened
- [ ]  The code is error free
- [ ]  The cells with code have been arranged by order of execution
- [ ]  The exploratory data analysis has been performed
- [ ]  The results of the exploratory data analysis are presented in the final notebook
- [ ]  The model's MAE score is not higher than 8
- [ ]  The model training code has been copied to the final notebook
- [ ]  The model training output has been copied to the final notebook
- [ ]  The findings have been provided based on the results of the model training